In [1]:
from llama_index.core import PromptTemplate
import json
import torch
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from pyvi import ViTokenizer
import weaviate
from weaviate.classes.init import Auth

/home/drissdo/Desktop/VN_LAW_QA_SYSTEM/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configuration constants
WEAVIATE_URL="https://mmtip1s2qwwe1q3bwih2fw.c0.us-west3.gcp.weaviate.cloud"
WEAVIATE_API_KEY="37Nj5DR1LWMDz98URsr3ib7e80pnHHAoyqn1"
DATA_COLLECTION = "ND168"
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
MODEL_NAME = "dangvantuan/vietnamese-document-embedding"
CHUNK_SIZE = 512  # Optimized for Vietnamese text
CHUNK_OVERLAP = 50  # Small overlap to maintain context

In [3]:
# Setup vector store
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_API_KEY),
)
vector_store = WeaviateVectorStore(
    weaviate_client=client,
    index_name=DATA_COLLECTION
)

In [ ]:
SYSTEM_PROMPT = PromptTemplate(
    template=(
                "Dựa trên tài liệu đã trích xuất, hãy phân tích và trả lời câu hỏi.\n"
                "Câu hỏi: {question}\n"
                "Tài liệu: {context}\n"
                "Hãy suy nghĩ từng bước:\n"
                "1. Phân tích xem thông tin có đủ và liên quan không?\n"
                "2. Nếu chưa đủ, hãy đưa ra truy vấn mới để tìm thêm thông tin\n"
                "3. Nếu đã đủ, đưa ra câu trả lời cuối cùng\n\n"
                "Hãy trả lời theo định dạng sau:\n"
                "Phân tích: <phân tích thông tin hiện có>\n"
                "Quyết định: [Cần thêm thông tin/Đã đủ thông tin]\n"
                "Truy vấn tiếp theo: <truy vấn mới> (nếu cần)\n"
                "Câu trả lời cuối cùng: <câu trả lời> (nếu đã đủ thông tin)\n"
            )
)

In [4]:
index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
    embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, device=DEVICE, trust_remote_code=True),
)

In [9]:
# Create retriever
retriever = index.as_retriever(
    vector_store_query_mode="hybrid",
    similarity_top_k=5,
    alpha=0.5  # Balance between keyword and semantic search
)

In [12]:
question = "nồng độ cồn "
retrieved_docs = retriever.retrieve(question)

for i, node in enumerate(retrieved_docs):
    print(f"Result {i+1}:\n")
    print(f"Text: {node.text}\n")
    print(f"Metadata: {node.metadata}\n")
    print(f"Score: {node.score}\n")
    print("="*50)

Result 1:

Text: loại phương_tiện : ô_tô mức phạt : phạt tiền từ 18.000.000 đồng đến 20.000.000 đồng_nội_dung vi_phạm : điều_khiển xe trên đường mà trong máu hoặc hơi thở có nồng_độ cồn vượt quá 50 miligam đến 80 miligam / 100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam / 1 lít khí thở

Metadata: {'category': 'ô tô', 'fine_amount': 'Phạt tiền từ 18.000.000 đồng đến 20.000.000 đồng', 'violation_type': 'khác', 'original_text': 'Loại phương tiện: ô tô\nMức phạt: Phạt tiền từ 18.000.000 đồng đến 20.000.000 đồng\nNội dung vi phạm: Điều khiển xe trên đường mà trong máu hoặc hơi thở có nồng độ cồn vượt quá 50 miligam đến 80 miligam/100 mililít máu hoặc vượt quá 0,25 miligam đến 0,4 miligam/1 lít khí thở'}

Score: 0.8931957483291626

Result 2:

Text: loại phương_tiện : ô_tô mức phạt : phạt tiền từ 6.000.000 đồng đến 8.000.000 đồng_nội_dung vi_phạm : điều_khiển xe trên đường mà trong máu hoặc hơi thở có nồng_độ cồn nhưng chưa vượt quá 50 miligam / 100 mililít máu hoặc chưa vượt quá 0